In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [3]:
def scrape_links(page_number):
    url = f"https://www.cnbcindonesia.com/indeks/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 12 links from page 1
Total Links: 12


In [5]:
def scrape_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        title_elem = soup.find('h1')
        if title_elem:
            title_text = title_elem.text.strip()
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"class": "author"})
        if author_elem:
            author_text = author_elem.get_text()
            author_text = author_text.split('-')[1].strip()
            author_text = author_text.split(',')[0].strip()
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('div', {"class": "date"})
        if date_elem:
            date_text = date_elem.text.strip()
        else:
            date_text = "Date not found"
        #     # Category berita
        category_elements = soup.find('ul', {"class": "breadcrumb"})
        if category_elements:
            category_text = category_elements.find_all('li')
            category_text= category_text[2].get_text()
        else:
            category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('div', {"class": "detail_text"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"
          
        nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
        if nama_berita_match:
            nama_berita = nama_berita_match.group(1)
        else:
             nama_berita = "Nama_berita not found"        

        results.append({'title': title_text,
                        'author' : author_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'nama_berita' : nama_berita,
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)

df.head(10)

,title,author,category,date,content,nama_berita,link
0,Pendaftaran PPPK Guru Kebutuhan Khusus Ditutup...,Tim Redaksi,Berita,29 September 2023 08:10,"Jakarta, CNBC Indonesia - Badan Kebijakan Nega...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230929074...
1,"Underpass HM Yamin Medan Dibangun, Target Sele...",rah,Berita,29 September 2023 08:10,"Jakarta, CNBC Indonesia - Wali Kota Medan Bobb...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230929075...
2,"BEI Suspensi Perdagangan Bursa BNC Sekuritas, ...",Romys Binekasri,Berita Market,29 September 2023 08:17,"Jakarta, CNBC Indonesia - PT Bursa Efek Indone...",cnbcindonesia,https://www.cnbcindonesia.com/market/202309290...
3,"Hati-hati Kalau Kehujanan, Awan Sudah Tercemar...",Mentari Puspa,Berita Lifestyle,29 September 2023 08:00,"Jakarta, CNBC Indonesia - Para peneliti di Jep...",cnbcindonesia,https://www.cnbcindonesia.com/lifestyle/202309...
4,"Duh! Harga Batu Bara Longsor 1,2% Gegara China",mza,Berita Research,29 September 2023 07:45,"Jakarta, CNBC Indonesia - Harga batu bara kemb...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023092...
5,"Breaking! Harga Emas Pegadaian Terjun Rp10,000",Robertus Andrianto,Berita MyMoney,29 September 2023 08:20,"Jakarta, CNBC Indonesia -Harga emas batangan y...",cnbcindonesia,https://www.cnbcindonesia.com/mymoney/20230929...
6,"Tetangga RI Terang-terangan Tantang China, AS ...",luc,Berita,29 September 2023 08:03,"Jakarta, CNBC Indonesia - Penyingkiran penghal...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230929080...
7,Simak Awal Musim Hujan di Jakarta & Wilayah RI...,Novina Putri Bestari,Berita Tech,29 September 2023 08:15,"Jakarta, CNBC Indonesia - Badan Meteorologi, K...",cnbcindonesia,https://www.cnbcindonesia.com/tech/20230929062...
8,"Percakapan Tentara Rusia Bocor, Kebohongan Put...",luc,Berita,29 September 2023 07:42,"Jakarta, CNBC Indonesia - Intelijen militer Uk...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230929073...
9,Asing Diam-diam Borong 10 Saham Ini Kala IHSG ...,Zefanya Aprilia,Berita Market,29 September 2023 08:05,"Jakarta, CNBC Indonesia - Indeks Harga Saham G...",cnbcindonesia,https://www.cnbcindonesia.com/market/202309290...


Save File

In [8]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/cnbc_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/cnbc_2023-09-29_08-22-34.xlsx
